In [1]:
import numpy as np
import pandas as pd
lottorydf = pd.read_csv('lottory.csv')
lottorydf=lottorydf.sort_values(by=['期號'],ascending=True)
lottorydf=lottorydf.reset_index(drop=True)
lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6','預測2區']] = lottorydf[['前1','前2','前3','前4','前5','前6','后區']].shift(-1)
print(lottorydf.tail())
print(lottorydf.shape)

         期號  前1  前2  前3  前4  前5  前6  后區   預測1   預測2   預測3   預測4   預測5   預測6  \
415  110085   6  10  13  16  29  33   3  10.0  18.0  32.0  27.0   3.0  14.0   
416  110086  10  18  32  27   3  14   7  37.0  16.0  32.0  10.0  19.0  22.0   
417  110087  37  16  32  10  19  22   6  38.0  11.0   5.0  37.0   9.0  35.0   
418  110088  38  11   5  37   9  35   4   6.0  33.0  30.0  22.0   8.0  18.0   
419  110089   6  33  30  22   8  18   4   NaN   NaN   NaN   NaN   NaN   NaN   

     預測2區  
415   7.0  
416   6.0  
417   4.0  
418   4.0  
419   NaN  
(420, 15)


In [2]:
# print(lottorydf.iloc[:,8:14]/38)
lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6']]=lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6']]/38
lottorydf[['前1','前2','前3','前4','前5','前6']]=lottorydf[['前1','前2','前3','前4','前5','前6']]/38
lottorydf[['后區','預測2區']]=lottorydf[['后區','預測2區']]/8
print(lottorydf.tail())

         期號        前1        前2        前3        前4        前5        前6  \
415  110085  0.157895  0.263158  0.342105  0.421053  0.763158  0.868421   
416  110086  0.263158  0.473684  0.842105  0.710526  0.078947  0.368421   
417  110087  0.973684  0.421053  0.842105  0.263158  0.500000  0.578947   
418  110088  1.000000  0.289474  0.131579  0.973684  0.236842  0.921053   
419  110089  0.157895  0.868421  0.789474  0.578947  0.210526  0.473684   

        后區       預測1       預測2       預測3       預測4       預測5       預測6   預測2區  
415  0.375  0.263158  0.473684  0.842105  0.710526  0.078947  0.368421  0.875  
416  0.875  0.973684  0.421053  0.842105  0.263158  0.500000  0.578947  0.750  
417  0.750  1.000000  0.289474  0.131579  0.973684  0.236842  0.921053  0.500  
418  0.500  0.157895  0.868421  0.789474  0.578947  0.210526  0.473684  0.500  
419  0.500       NaN       NaN       NaN       NaN       NaN       NaN    NaN  


In [40]:
def preparedata(df, ref_day, predict_day):
    X_train, Y_train = [], []
    i = 0
    while i < (df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,1:8]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day,8:15]))
        i=i+ref_day+predict_day
    return np.array(X_train), np.array(Y_train)

In [80]:
X_train,Y_train= preparedata(lottorydf[:lottorydf.shape[0]-24],8,1)
print(X_train.shape,len(X_train),Y_train.shape,len(Y_train),X_train.ndim)
X_test,Y_test = preparedata(lottorydf[lottorydf.shape[0]-40:],8,1)
print(X_test.shape,len(X_test),Y_test.shape,len(Y_test),X_test.ndim)

(44, 8, 7) 44 (44, 1, 7) 44 3
(4, 8, 7) 4 (4, 1, 7) 4 3


In [42]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
import tensorflow as tf

In [79]:
batchsz = 4
data = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
data = data.batch(batchsz,drop_remainder=True)


data_test = tf.data.Dataset.from_tensor_slices((X_test,Y_test))
data_test = data_test.batch(batchsz,drop_remainder=True)
# list(data_test.as_numpy_iterator())

In [74]:
model = Sequential()
model.add(LSTM(4,batch_input_shape=(8,7), return_sequences=False, stateful=True,))
model.add(Dense(1))
model.add(Activation('relu'))
model.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['accuracy'])
model.summary()

ValueError: Input 0 of layer lstm_35 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (8, 7)

In [63]:
model.fit(data, epochs=10)


Epoch 1/10
11/11 [==============================] - 3s 23ms/step - loss: 0.1847 - accuracy: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 0s 26ms/step - loss: 0.1149 - accuracy: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 0s 26ms/step - loss: 0.0840 - accuracy: 0.0134
Epoch 4/10
11/11 [==============================] - 0s 27ms/step - loss: 0.0790 - accuracy: 0.0540
Epoch 5/10
11/11 [==============================] - 0s 26ms/step - loss: 0.0802 - accuracy: 0.0580
Epoch 6/10
11/11 [==============================] - 0s 23ms/step - loss: 0.0797 - accuracy: 0.0544
Epoch 7/10
11/11 [==============================] - 0s 25ms/step - loss: 0.0794 - accuracy: 0.0487
Epoch 8/10
11/11 [==============================] - 0s 23ms/step - loss: 0.0793 - accuracy: 0.0459
Epoch 9/10
11/11 [==============================] - 0s 30ms/step - loss: 0.0792 - accuracy: 0.0479
Epoch 10/10
11/11 [==============================] - 0s 29ms/step - loss: 0.0792 - accuracy: 0.0491


In [64]:
predicts = model.predict(data_test,batch_size=4)
print(type(predicts))
print(predicts)

<class 'numpy.ndarray'>
[[[0.5215833 ]
  [0.5217101 ]
  [0.56219685]
  [0.5545999 ]
  [0.51357555]
  [0.5471431 ]
  [0.5951677 ]
  [0.55577296]]

 [[0.51493245]
  [0.5149806 ]
  [0.5288744 ]
  [0.53336716]
  [0.51959413]
  [0.49911183]
  [0.49478444]
  [0.5021569 ]]

 [[0.5665976 ]
  [0.5290045 ]
  [0.5514489 ]
  [0.55770916]
  [0.5298516 ]
  [0.49446115]
  [0.4980474 ]
  [0.51211536]]

 [[0.5139653 ]
  [0.5040624 ]
  [0.5491411 ]
  [0.52869254]
  [0.5042917 ]
  [0.49815622]
  [0.50278556]
  [0.5307876 ]]]


In [66]:
print(predicts.shape)

(4, 8, 1)
